In [1]:
from blocks import UnetEncodeLayer,UnetUpscaleLayer,UnetForwardDecodeLayer, conv3x3
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms.v2 as v2
import torchvision.transforms.functional as functional
import math
from transformers import AutoModel, AutoImageProcessor, SegformerForSemanticSegmentation, SegformerConfig
from blocks import UnetEncodeLayer,UnetUpscaleLayer,UnetForwardDecodeLayer, VisionTransformerEncoder
from torchvision.models.segmentation import deeplabv3_resnet101,deeplabv3_mobilenet_v3_large

C:\Users\eros\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class DeepLabV3MobileNet_test(nn.Module):
	def __init__(self, num_classes, pretrained=True):
		super(DeepLabV3MobileNet_test, self).__init__()		
		self.requires_context = False
		self.wrapper = True
		self.returns_logits = True		
		self.num_classes = num_classes

		if pretrained:
			self.model = deeplabv3_mobilenet_v3_large( weights=torchvision.models.segmentation.DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT)
			in_channels = self.model.classifier[4].in_channels
			print(self.model.classifier)
			self.model.classifier[4] = torch.nn.Conv2d(in_channels, self.num_classes, kernel_size=1)
			print(self.model.classifier)
		else:
			self.model = deeplabv3_mobilenet_v3_large(self.num_classes)
	def forward(self, x: torch.Tensor, context=None):
		d = self.model(x)
		return d['out']
	def custom_load(self, checkpoint):
		checkpoint_state_dict_mod = {}
		checkpoint_state_dict = checkpoint['model_state_dict']
		for item in checkpoint_state_dict:
			checkpoint_state_dict_mod[str(item).replace('module.', '')] = checkpoint_state_dict[item]
		self.model.load_state_dict(checkpoint_state_dict_mod)		

In [9]:
net = DeepLabV3MobileNet_test(16)

DeepLabHead(
  (0): ASPP(
    (convs): ModuleList(
      (0): Sequential(
        (0): Conv2d(960, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (1): ASPPConv(
        (0): Conv2d(960, 256, kernel_size=(3, 3), stride=(1, 1), padding=(12, 12), dilation=(12, 12), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (2): ASPPConv(
        (0): Conv2d(960, 256, kernel_size=(3, 3), stride=(1, 1), padding=(24, 24), dilation=(24, 24), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (3): ASPPConv(
        (0): Conv2d(960, 256, kernel_size=(3, 3), stride=(1, 1), padding=(36, 36), dilation=(36, 36), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_runn